# Community Analyst Data Translator
Tabular reports built in legacy Community Analyst can be exported (credit consumption) into excel files, which in turn can be normalized and turned into tables. This information is generated by running the report on a given geography within analyst, so it can be tied into a dashboard with ease.

### dependencies

In [1]:
import pandas as pd
import os
import warnings
warnings.simplefilter("ignore", category=UserWarning)
import xlsxwriter

### path
in the CA report, theres 12 different tables, definition to capture each separate table

In [7]:
# Initialize empty dataframes for each table with proper column names
description_table = pd.DataFrame(columns=['Description', 'Value', 'OD_ID'])
retail_spending_summary = pd.DataFrame(columns=['NAICS Code', 'Industry Summary', 'Spending Potential Index', 'Average Spent', 'Total', 'OD_ID'])
spending_summary_by_naics = pd.DataFrame(columns=['NAICS Code', 'Industry Subsector & Group', 'Index', 'Average Spending', 'Total', 'OD_ID'])
key_demographic_indicators = pd.DataFrame(columns=['Indicator', 'Value', 'OD_ID'])
journey_to_work = pd.DataFrame(columns=['Journey Type', 'Value', 'OD_ID'])
vehicles_available = pd.DataFrame(columns=['Vehicle Availability', 'Value', 'OD_ID'])
income_table = pd.DataFrame(columns=['Income Range', 'Number of Households', 'OD_ID'])
housing_and_households = pd.DataFrame(columns=['Housing Category', 'Value', 'OD_ID'])
income_tier_per_household = pd.DataFrame(columns=['Income Tier', 'Number of Households', 'OD_ID'])
tapestry_segments = pd.DataFrame(columns= ['Segment Code', 'Area Value','Harris County', 'Texas', 'United States','OD_ID'])
tapestry_lifemode = pd.DataFrame(columns= ['Tapestry Lifemode', 'Households','HHs %', '% US HHs', 'Index', 'OD_ID'])
key_demographic_by_generation = pd.DataFrame(columns= ['Generation', 'Area Population', 'Harris County', 'OD_ID'])

In [3]:
def extract_tables(file_path):
    # Load the Excel file
    excel_data = pd.ExcelFile(file_path)

    # Load the sheet
    sheet_name = excel_data.sheet_names[0]
    df = excel_data.parse(sheet_name)

    # Extract the OD_ID from the file name (extracting the number only)
    od_id = ''.join(filter(str.isdigit, file_path.split('/')[-1].split('.')[0]))

    # Table 1: Description Table
    description_table = pd.DataFrame({
        'Description': ['Title', 'OD Area', 'OD_ID', 'Area (square miles)'],
        'Value': [
            df.iloc[0, 0],
            df.iloc[1, 0],
            od_id,
            ''.join(c for c in df.iloc[2, 0].split(':')[-1].strip() if c.isdigit() or c == '.')
        ]
    })
    description_table['OD_ID'] = od_id

    # Helper function to add OD_ID column
    def add_od_id(table):
        table['OD_ID'] = od_id
        return table

    # Table 2: Retail Spending Summary
    retail_spending_summary = df.iloc[5:8, 0:5]
    retail_spending_summary.columns = ['NAICS Code', 'Industry Summary', 'Spending Potential Index', 'Average Spent', 'Total']
    retail_spending_summary = add_od_id(retail_spending_summary)

    # Table 3: Spending Summary by NAICS
    spending_summary_by_naics = df.iloc[10:37, 0:5]
    spending_summary_by_naics.columns = ['NAICS Code', 'Industry Subsector & Group', 'Index', 'Average Spending', 'Total']
    spending_summary_by_naics = add_od_id(spending_summary_by_naics)

    # Table 4: Key Demographic Indicators
    key_demographic_indicators = df.iloc[39:46, 0:2]
    key_demographic_indicators.columns = ['Indicator', 'Value']
    key_demographic_indicators = add_od_id(key_demographic_indicators)

    # Table 5: Journey to Work
    journey_to_work = df.iloc[48:52, 0:2]
    journey_to_work.columns = ['Journey Type', 'Value']
    journey_to_work = add_od_id(journey_to_work)

    # Table 6: Vehicles Available in Households
    vehicles_available = df.iloc[54:60, 0:2]
    vehicles_available.columns = ['Vehicle Availability', 'Value']
    vehicles_available = add_od_id(vehicles_available)

    # Table 7: Income
    income_table = df.iloc[62:65, 0:2]
    income_table.columns = ['Income Range', 'Number of Households']
    income_table = add_od_id(income_table)

    # Table 8: Housing and Households
    housing_and_households = df.iloc[67:72, 0:2]
    housing_and_households.columns = ['Housing Category', 'Value']
    housing_and_households = add_od_id(housing_and_households)

    # Table 9: Income Tier per Household
    income_tier_per_household = df.iloc[76:79, 0:2]
    income_tier_per_household.columns = ['Income Tier', 'Number of Households']
    income_tier_per_household = add_od_id(income_tier_per_household)

    # Table 10: Tapestry Segments
    tapestry_segments = df.iloc[84:118, 0:5]
    tapestry_segments.columns = ['Segment Code', 'Area Value','Harris County', 'Texas', 'United States']
    tapestry_segments = add_od_id(tapestry_segments)

    # Table 11: Tapestry Lifemode
    tapestry_lifemode = df.iloc[121:136, 0:5]
    tapestry_lifemode.columns = ['Tapestry Lifemode', 'Households','HHs %', '% US HHs', 'Index']
    tapestry_lifemode = add_od_id(tapestry_lifemode)

    # Table 12: Key Demographic by Generation
    key_demographic_by_generation = df.iloc[138:146, 0:3]
    key_demographic_by_generation.columns = ['Generation', 'Area Population', 'Harris County']
    key_demographic_by_generation = add_od_id(key_demographic_by_generation)

    return {
        'Description Table': description_table,
        'Retail Spending Summary': retail_spending_summary,
        'Spending Summary by NAICS': spending_summary_by_naics,
        'Key Demographic Indicators': key_demographic_indicators,
        'Journey to Work': journey_to_work,
        'Vehicles Available': vehicles_available,
        'Income Table': income_table,
        'Housing and Households': housing_and_households,
        'Income Tier per Household': income_tier_per_household,
        'Tapestry Segments': tapestry_segments,
        'Tapestry Lifemode': tapestry_lifemode,
        'Key Demographic by Generation': key_demographic_by_generation
    }

### TEST 1

In [5]:
test_path = '../ltsv_esrica/3.xlsx'

In [6]:
try:
    tables = extract_tables(test_path)
    
    # Append data to the pre-initialized dataframes
    description_table = pd.concat([description_table, tables['Description Table']], ignore_index=True)
    retail_spending_summary = pd.concat([retail_spending_summary, tables['Retail Spending Summary']], ignore_index=True)
    spending_summary_by_naics = pd.concat([spending_summary_by_naics, tables['Spending Summary by NAICS']], ignore_index=True)
    key_demographic_indicators = pd.concat([key_demographic_indicators, tables['Key Demographic Indicators']], ignore_index=True)
    journey_to_work = pd.concat([journey_to_work, tables['Journey to Work']], ignore_index=True)
    vehicles_available = pd.concat([vehicles_available, tables['Vehicles Available']], ignore_index=True)
    income_table = pd.concat([income_table, tables['Income Table']], ignore_index=True)
    housing_and_households = pd.concat([housing_and_households, tables['Housing and Households']], ignore_index=True)
    income_tier_per_household = pd.concat([income_tier_per_household, tables['Income Tier per Household']], ignore_index=True)
    tapestry_segments = pd.concat([tapestry_segments, tables['Tapestry Segments']], ignore_index=True)
    tapestry_lifemode = pd.concat([tapestry_lifemode, tables['Tapestry Lifemode']], ignore_index=True)
    key_demographic_by_generation = pd.concat([key_demographic_by_generation, tables['Key Demographic by Generation']], ignore_index=True)
except Exception as e:
    print(f"Error processing file {test_path}: {str(e)}")

Error processing file ../ltsv_esrica/3.xlsx: [Errno 2] No such file or directory: '../ltsv_esrica/3.xlsx'


### full export

In [9]:
directory_path = './ltsv_esrica'

In [11]:
for file_path in [os.path.join(directory_path, f) for f in os.listdir(directory_path) if f.endswith('.xlsx')]:
    try:
        tables = extract_tables(file_path)
        
        # Append data to the pre-initialized dataframes
        description_table = pd.concat([description_table, tables['Description Table']], ignore_index=True)
        retail_spending_summary = pd.concat([retail_spending_summary, tables['Retail Spending Summary']], ignore_index=True)
        spending_summary_by_naics = pd.concat([spending_summary_by_naics, tables['Spending Summary by NAICS']], ignore_index=True)
        key_demographic_indicators = pd.concat([key_demographic_indicators, tables['Key Demographic Indicators']], ignore_index=True)
        journey_to_work = pd.concat([journey_to_work, tables['Journey to Work']], ignore_index=True)
        vehicles_available = pd.concat([vehicles_available, tables['Vehicles Available']], ignore_index=True)
        income_table = pd.concat([income_table, tables['Income Table']], ignore_index=True)
        housing_and_households = pd.concat([housing_and_households, tables['Housing and Households']], ignore_index=True)
        income_tier_per_household = pd.concat([income_tier_per_household, tables['Income Tier per Household']], ignore_index=True)
        tapestry_segments = pd.concat([tapestry_segments, tables['Tapestry Segments']], ignore_index=True)
        tapestry_lifemode = pd.concat([tapestry_lifemode, tables['Tapestry Lifemode']], ignore_index=True)
        key_demographic_by_generation = pd.concat([key_demographic_by_generation, tables['Key Demographic by Generation']], ignore_index=True)
    except Exception as e:
        print(f"Error processing file {test_path}: {str(e)}")

#### some cleaning

In [12]:
tapestry_lifemode = tapestry_lifemode[tapestry_lifemode['Index'].notna()]


### export each table as a tab into one excel file

In [13]:
# Function to save all tables to a single Excel file with separate sheets
def save_to_excel(output_path):
    with pd.ExcelWriter(output_path, engine='xlsxwriter') as writer:
        for table_name, table_df in {
            'Description Table': description_table,
            'Retail Spending Summary': retail_spending_summary,
            'Spending Summary by NAICS': spending_summary_by_naics,
            'Key Demographic Indicators': key_demographic_indicators,
            'Journey to Work': journey_to_work,
            'Vehicles Available': vehicles_available,
            'Income Table': income_table,
            'Housing and Households': housing_and_households,
            'Income Tier per Household': income_tier_per_household,
            'Tapestry Segments': tapestry_segments,
            'Tapestry Lifemode': tapestry_lifemode,
            'Key Demographic by Generation': key_demographic_by_generation
        }.items():
            table_df.to_excel(writer, sheet_name=table_name, index=False, startrow=1, header=False)
            workbook = writer.book
            worksheet = writer.sheets[table_name]

            # Get the column names and the number of rows and columns
            columns = [{'header': col} for col in table_df.columns]
            num_rows, num_cols = table_df.shape

            # Add the table with formatting
            worksheet.add_table(0, 0, num_rows, num_cols - 1, {
                'columns': columns,
                'name': table_name.replace(' ', '_'),
                'autofilter': True,
                'style': 'Table Style Medium 9'
            })
    print(f"All tables have been saved to {output_path}")

In [15]:
# Call the function to save the data to an Excel file
output_path = './pnr_esrigc/community_analyst_export.xlsx'
save_to_excel(output_path)

All tables have been saved to ./pnr_esrigc/community_analyst_export.xlsx
